In [1]:
import numpy as np
import os.path as osp
import pandas as pd
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors  # Students: you can use this implementation to find the 
                                                # Nearest-Neigbors
import sys
sys.path.append('../..')
from cs233_gtda_hw4.losses import EncodingDistance

In [2]:
# Load golden distances (pairwise matrix, or corresponding model/part names in golden_names)
golden_part_dist_file = '../data/golden_dists.npz'
golden_data = np.load(golden_part_dist_file, allow_pickle=True)
golden_part_dist = golden_data['golden_part_dist']
golden_names = golden_data['golden_names']
print(len(golden_names))  # models-name/part combinations
print(golden_names[0])

484
c59cdd1537bd75ddd0818327fc390a5__2__


In [3]:
# Load/organize golden part-aware distances.
sn_id_to_parts = defaultdict(list)
id_to_part_loc = dict()

for i, name in enumerate(golden_names):
    # Extract shape-net model ids of golden, map them to their parts.
    sn_id, _, part_id, _, _ = name.split('_')
    sn_id_to_parts[sn_id].append(part_id)
    
    # Map shape-net model id and part_id to location in distance matrix, (the order is the same).
    id_to_part_loc[(sn_id, part_id)] = i

In [4]:
out_n = 1024
enc_dist = EncodingDistance('../data/golden_dists.npz')

def measure_part_awareness(model_tag):
    if model_tag != 'random':
        ae_emb_file = f'../data/out/{model_tag}_latent_codes.npz'
        in_d = np.load(ae_emb_file)

        latent_codes = in_d['latent_codes']
        test_names = in_d['test_names']
    else:
        latent_codes = np.random.rand(len(test_names), 123)

    res = enc_dist.calculate(latent_codes, test_names)
    res['model_tag'] = model_tag
    return res

    # print(f'''
    #     {model_tag} PART AWARENESS: \n
    #     Cumulative encoding distance = {encoding_distances.sum():.4f} 
    #     Avg. shared parts = {num_shared_parts.mean():.2f} 
    #     Avg. latent distance = {latent_distances.mean():.2f}    
    # ''')
    
    # return {
    #     'model_tag': model_tag,
    #     'enc_dist': encoding_distances.sum(),
    #     'shared_pts': num_shared_parts.mean(),
    #     'latent_dist': latent_distances.mean()
    # }

In [5]:
data = []
tags = [f'pc_ae_out{out_n}_', f'part_pc_ae_out{out_n}_', f'experiment_',
        f'experiment_drop0.15_bnorm_',
        'experiment_drop0.15_bnorm_decay0.8_alpha1e-5_',
       'experiment_drop0.15_bnorm_decay0.8_alpha1e-6_',
       'experiment_drop0.15_bnorm_decay0.7_alpha1e-6_',
       'experiment_kl5e-5_noisy_', 'experiment_kl5e-5_noisy2e-2_',
        'experiment_drop0.15_bnorm_kl5e-5_noisy2e-2_', 
        'experiment_drop0.2_bnorm_kl5e-05_noisy0.005_cdec0.7_alpha1e-06_',
        'experiment_drop0.2_bnorm_kl1e-05_noisy0.005_cdec0.7_alpha1e-06_kdec1.1_',
        'experiment_drop0.15_bnorm_kl1e-05_noisy0.01_cdec0.7_alpha1e-05_kdec1_',
        'experiment_drop0.15_bnorm_kl1e-05_noisy0.001_cdec0.7_alpha1e-05_kdec1_',
        # 'experiment_drop0.2_bnorm_kl1e-05_noisy0.005_cdec0.85_alpha1e-05_kdec1.4_', # 412
        # 'experiment_drop0.2_bnorm_kl1e-05_noisy0.005_cdec0.85_alpha1e-06_kdec1.4_', $ 405
       ]
for model_tag in tags:
    data.append(measure_part_awareness(model_tag))
pd.DataFrame(data)

enc_dist  shared_pts  latent_dist  \
0   406.436414    3.146667     0.213306   
1   417.841077    3.186667     0.266832   
2   412.147745    3.146667     0.229384   
3   382.428042    3.173333     0.556023   
4   405.770041    3.160000     0.531155   
5   389.672346    3.160000     0.649659   
6   390.321417    3.166667     0.681365   
7   402.218711    3.153333     1.740098   
8   407.687951    3.153333     0.895510   
9   408.786409    3.160000     2.316503   
10  387.446824    3.153333     5.582441   
11  385.248844    3.120000     5.367769   
12  429.341322    3.080000     2.082850   
13  413.226048    3.173333     5.095382   

                                            model_tag  
0                                      pc_ae_out1024_  
1                                 part_pc_ae_out1024_  
2                                         experiment_  
3                          experiment_drop0.15_bnorm_  
4       experiment_drop0.15_bnorm_decay0.8_alpha1e-5_  
5       experiment_drop0.15_bnorm_decay0.8_alpha1e-6_  
6       experiment_drop0.15_bnorm_decay0.7_alpha1e-6_  
7                            experiment_kl5e-5_noisy_  
8                        experiment_kl5e-5_noisy2e-2_  
9         experiment_drop0.15_bnorm_kl5e-5_noisy2e-2_  
10  experiment_drop0.2_bnorm_kl5e-05_noisy0.005_cd...  
11  experiment_drop0.2_bnorm_kl1e-05_noisy0.005_cd...  
12  experiment_drop0.15_bnorm_kl1e-05_noisy0.01_cd...  
13  experiment_drop0.15_bnorm_kl1e-05_noisy0.001_c...

In [8]:
# BEST POSSIBLE PART DISTANCES
ae_emb_file = f'../data/out/{model_tag}_latent_codes.npz'
in_d = np.load(ae_emb_file)

latent_codes = in_d['latent_codes']
test_names = in_d['test_names']
encoding_distances = np.zeros(len(test_names))
for i, sn_name in enumerate(test_names):
    parts_of_model = set(sn_id_to_parts[sn_name])

    min_distance = np.inf
    # greedily search for the best possible match
    for matched_neighbor in test_names:
        if matched_neighbor == sn_name:
            continue
        distance = 0
        parts_of_neighbor = set(sn_id_to_parts[matched_neighbor])

        # compute the requested distances.
        # Use id_to_part_loc for each model/part combination
        parts_in_both = parts_of_model.intersection(parts_of_neighbor)
        for k in parts_in_both:
            distance += golden_part_dist[id_to_part_loc[(sn_name, k)], id_to_part_loc[(matched_neighbor, k)]]

        parts_only_model = parts_of_model.difference(parts_of_neighbor)
        parts_only_neighbor = parts_of_neighbor.difference(parts_of_model)
        # for k in parts_only_A:
        #     distance += max([golden_part_dist[id_to_part_loc[(matched_neighbor, k)], id_to_part_loc[(sn_name, u)]] for u in parts_of_model])

        # part distance as in homework
        cand_distances = [0] * 4
        for u in parts_of_model:
            for k in parts_only_neighbor:
                cand_distances[int(u) - 1] += golden_part_dist[
                    id_to_part_loc[(matched_neighbor, k)], id_to_part_loc[(sn_name, u)]]
        distance += max(cand_distances)
        
        if distance < min_distance:
            min_distance = distance
    encoding_distances[i] = min_distance

print(f'''
    IDEAL PART AWARENESS: \n
    Cumulative encoding distance = {encoding_distances.sum():.4f}
''')


    IDEAL PART AWARENESS: 

    Cumulative encoding distance = 322.3948

